# Live Model Server Testing

Test model server via HTTP calls 

### Deploy model server for testing

In [1]:
import os
import mlrun

project_name = "sk-project"
DATA_PATH = "https://s3.wasabisys.com/iguazio/data/iris/iris_dataset.csv"
MODEL_PATH = "https://s3.wasabisys.com/iguazio/models/iris/model.pkl"

_, artifact_path = mlrun.set_environment(
    api_path="http://mlrun-api:8080", artifact_path=os.path.abspath("./")
)

fn = mlrun.import_function("hub://v2_model_server")
fn.add_model("mymodel", model_path=MODEL_PATH)
address = fn.deploy()

> 2021-02-17 15:07:57,298 [info] Starting remote function deploy
2021-02-17 15:07:57  (info) Deploying function
2021-02-17 15:07:57  (info) Building
2021-02-17 15:07:57  (info) Staging files and preparing base images
2021-02-17 15:07:57  (info) Building processor image
2021-02-17 15:07:58  (info) Build complete
2021-02-17 15:08:04  (info) Function deploy complete
> 2021-02-17 15:08:05,100 [info] function deployed, address=default-tenant.app.function-marketpace.iguazio-cd0.com:32440


### Run model server tester locally

In [2]:
from v2_model_tester import model_server_tester

gen = mlrun.run_local(
    name="model_server_tester",
    handler=model_server_tester,
    params={"addr": address, "model": "mymodel"},
    inputs={"table": DATA_PATH},
    project=project_name,
    artifact_path=os.path.join(artifact_path, "data"),
)

> 2021-02-17 15:08:05,115 [info] starting run model_server_tester uid=6dab26b75bf64a6c985e2be178691205 DB=http://mlrun-api:8080
> 2021-02-17 15:08:05,324 [info] testing with dataset against http://default-tenant.app.function-marketpace.iguazio-cd0.com:32440, model: mymodel
{'id': '7623b52d-fada-43ce-baa4-cc91606911b1', 'model_name': 'mymodel', 'outputs': [2]}
{'id': '207d4ce9-d059-4ae9-981f-184eb7466a65', 'model_name': 'mymodel', 'outputs': [2]}
{'id': '1e73cadc-c0a0-4797-9f7f-3b6b53575c44', 'model_name': 'mymodel', 'outputs': [0]}
{'id': 'df8838bf-b639-4d0c-b49d-fe61ce52bd19', 'model_name': 'mymodel', 'outputs': [2]}
{'id': '8ba46cdd-f32a-4018-a0ee-8fcba54e7c91', 'model_name': 'mymodel', 'outputs': [1]}
{'id': 'c5486ae5-3385-47e4-a6fe-c9524e8cac4c', 'model_name': 'mymodel', 'outputs': [2]}
{'id': '2953a591-e0ea-474d-a01b-ae3752408222', 'model_name': 'mymodel', 'outputs': [2]}
{'id': 'b98f2d8b-dc73-4abd-80ff-da62c7d07764', 'model_name': 'mymodel', 'outputs': [2]}
{'id': 'a4110552-020e-

project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
sk-project,...78691205,0,Feb 17 15:08:05,completed,model_server_tester,v3io_user=eyalskind=handlerowner=eyalshost=jupyter-eyals-666bf556fc-5v7bf,table,addr=http://default-tenant.app.function-marketpace.iguazio-cd0.com:32440model=mymodel,total_tests=20errors=0match=5avg_latency=27300min_latency=26189max_latency=28732,latency


to track results use .show() or .logs() or in CLI: 
!mlrun get run 6dab26b75bf64a6c985e2be178691205 --project sk-project , !mlrun logs 6dab26b75bf64a6c985e2be178691205 --project sk-project
> 2021-02-17 15:08:05,992 [info] run executed, status=completed


### Save

In [3]:
import yaml

with open("item.yaml") as item_file:
    items = yaml.load(item_file, Loader=yaml.FullLoader)

test_func = mlrun.code_to_function(
    items["name"],
    kind=items["spec"]["kind"],
    handler=items["spec"]["handler"],
    filename=items["spec"]["filename"],
    image=items["spec"]["image"],
    description=items["description"],
    categories=items["categories"],
    labels=items["labels"],
    requirements=items["spec"]["requirements"],
)

test_func.export("v2_model_tester.yaml")

> 2021-02-17 15:08:06,087 [info] function spec saved to path: v2_model_tester.yaml


### Run remotely

In [4]:
test_func.run(
    mlrun.NewTask(
        name="model_server_tester",
        handler=model_server_tester,
        params={"addr": address, "model": "mymodel"},
        inputs={"table": DATA_PATH},
        project=project_name,
        artifact_path=os.path.join(artifact_path, "data"),
    )
)

> 2021-02-17 15:08:06,095 [info] starting run model_server_tester uid=6dc53ded53694f4596b7bee3a7410844 DB=http://mlrun-api:8080
> 2021-02-17 15:08:06,232 [info] Job is running in the background, pod: model-server-tester-q66gm
> 2021-02-17 15:08:08,893 [info] testing with dataset against http://default-tenant.app.function-marketpace.iguazio-cd0.com:32440, model: mymodel
{'id': 'f1c763e6-550c-4854-bfbc-2ec176a95d76', 'model_name': 'mymodel', 'outputs': [0]}
{'id': '92889765-dea0-46c9-bfb2-2f0979fe6bbb', 'model_name': 'mymodel', 'outputs': [0]}
{'id': '6ac12146-cbed-4cd8-a4e2-0280d2f15af0', 'model_name': 'mymodel', 'outputs': [1]}
{'id': '53bc5511-9ec5-4f4b-a33b-d377593c4188', 'model_name': 'mymodel', 'outputs': [0]}
{'id': 'c2840940-a04d-4726-a7f9-aa6dc2ad96b6', 'model_name': 'mymodel', 'outputs': [0]}
{'id': '6de2301a-cfe4-4e74-a292-636bda3362f8', 'model_name': 'mymodel', 'outputs': [2]}
{'id': '73c49c2d-b5a7-4a23-97b8-f14719990b83', 'model_name': 'mymodel', 'outputs': [1]}
{'id': '1dcc

project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
sk-project,...a7410844,0,Feb 17 15:08:08,completed,model_server_tester,v3io_user=eyalskind=jobowner=eyalshost=model-server-tester-q66gm,table,addr=http://default-tenant.app.function-marketpace.iguazio-cd0.com:32440model=mymodel,total_tests=20errors=0match=9avg_latency=25338min_latency=24835max_latency=28219,latency


to track results use .show() or .logs() or in CLI: 
!mlrun get run 6dc53ded53694f4596b7bee3a7410844 --project sk-project , !mlrun logs 6dc53ded53694f4596b7bee3a7410844 --project sk-project
> 2021-02-17 15:08:12,352 [info] run executed, status=completed
